<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/open_source_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup


In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!export LC_ALL=en_US.UTF-8;
%pip install openai
%pip install tiktoken
%pip install scikit-learn
%pip install matplotlib
%pip install plotly
%pip install pandas
%pip install llama-index
%pip install langchain
%pip install backoff
%pip install transformers
%pip install sentencepiece
%pip install wolframalpha
%pip install spacy
%pip install sentence_transformers
%pip install faiss-gpu
%pip install icetk
%pip install cpm_kernels

import openai, os
import faiss
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTFaissIndex, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import SpacyTextSplitter, CharacterTextSplitter
from langchain.chat_models import ChatOpenAI



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

# read Google Drive File

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
from googleapiclient.discovery import build
from llama_index.node_parser import SimpleNodeParser
from llama_index import Document, GPTSimpleVectorIndex
import os
openai.api_key_path = None
openai.api_key = os.getenv("OPENAI_API_KEY")

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)


drive_service = build('drive', 'v3')

## faq txt文档
data_file_id = "1ywpphG-DAXgYt0adWGW54jYdFDgf5loO"
data_file = drive_service.files().get(fileId=data_file_id).execute()
content = drive_service.files().get_media(fileId=data_file_id).execute().decode()
# print(content)

# documents = SimpleDirectoryReader('./data/faq/').load_data()
documents = Document(content)
nodes = parser.get_nodes_from_documents([documents])

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))

service_context = ServiceContext.from_defaults(embed_model=embed_model)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)
index = GPTFaissIndex(nodes=nodes,faiss_index=faiss_index, service_context=service_context)

ValidationError: ignored

# New Section

In [6]:
from llama_index import QueryMode

openai.api_key = os.environ.get("OPENAI_API_KEY")

response = index.query(
    "请问你们海南能发货吗？", 
    mode=QueryMode.EMBEDDING,
    verbose=True, 
)
print(response)

NameError: ignored